In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from sklearn.cross_validation import train_test_split,KFold
import numpy as np


c:\program files\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
datafile_train=r'C:\Users\id832393\Documents\Personal\Data science\Python\Projects\Project1\Data\Consumer_Complaints_train.csv'
datafile_test=r'C:\Users\id832393\Documents\Personal\Data science\Python\Projects\Project1\Data\Consumer_Complaints_test_share.csv'
cd_train=pd.read_csv(datafile_train)
cd_test=pd.read_csv(datafile_test)

print(cd_train.shape)
print(cd_test.shape)

cd_test.columns

cd_train.columns

cd_train.dtypes

#add response column in test data frame
cd_test['Consumer disputed?']=np.nan

#add flag column for test or train data indication
cd_train['data']='train'
cd_test['data']='test'

#aling column sequence in both dataframe
cd_test=cd_test[cd_train.columns]

#combine both dataset
cd_all = pd.concat([cd_train, cd_test], axis=0)

cd_all.head()

#gives type and unique values in each column.
list(zip(cd_all.columns, cd_all.dtypes, cd_all.nunique()))

In [3]:
#convert the columns into date time
for col in ['Date received','Date sent to company']:
    cd_train[col]=pd.to_datetime(cd_train[col],infer_datetime_format=True)
    cd_test[col]=pd.to_datetime(cd_test[col],infer_datetime_format=True)

In [4]:
#add new column day difference, derived from available columns
cd_train['day_diff']=pd.to_numeric(cd_train['Date sent to company']-cd_train['Date received'])
cd_test['day_diff']=pd.to_numeric(cd_test['Date sent to company']-cd_test['Date received'])

In [5]:
for col in ['Date received','Date sent to company']:
    cd_train.drop([col],1,inplace=True)
    cd_test.drop([col],1,inplace=True)

In [6]:
print(cd_train.shape)
print(cd_test.shape)
#print(cd_all.shape)

(478421, 17)
(119606, 16)


In [7]:
cd_train.dtypes

Product                         object
Sub-product                     object
Issue                           object
Sub-issue                       object
Consumer complaint narrative    object
Company public response         object
Company                         object
State                           object
ZIP code                        object
Tags                            object
Consumer consent provided?      object
Submitted via                   object
Company response to consumer    object
Timely response?                object
Consumer disputed?              object
Complaint ID                     int64
day_diff                         int64
dtype: object

In [8]:
cd_train.isnull().sum()

Product                              0
Sub-product                     138473
Issue                                0
Sub-issue                       292625
Consumer complaint narrative    403327
Company public response         388029
Company                              0
State                             3839
ZIP code                          3848
Tags                            411215
Consumer consent provided?      342934
Submitted via                        0
Company response to consumer         0
Timely response?                     0
Consumer disputed?                   0
Complaint ID                         0
day_diff                             0
dtype: int64

In [9]:
for col in cd_train.select_dtypes(['object']).columns:
    print(col,':',cd_train[col].nunique())

Product : 12
Sub-product : 47
Issue : 95
Sub-issue : 68
Consumer complaint narrative : 74019
Company public response : 10
Company : 3276
State : 62
ZIP code : 25962
Tags : 3
Consumer consent provided? : 4
Submitted via : 6
Company response to consumer : 7
Timely response? : 2
Consumer disputed? : 2


In [10]:
cd_train['Consumer consent provided?'].unique()

array([nan, 'Consent provided', 'Other', 'Consent not provided',
       'Consent withdrawn'], dtype=object)

Sub-product                     138473
Sub-issue                       292625
Consumer complaint narrative    403327
Company public response         388029
State                             3839
ZIP code                          3848
Tags                            411215
Consumer consent provided?      342934

In [11]:
len(pd.isnull(cd_train['Tags']))
#len(cd_train)

478421

In [12]:
for col in ['Sub-product','Sub-issue','Consumer complaint narrative',
            'Company public response','Tags','Consumer consent provided?']:
    varname=col.replace('-','_').replace('?','').replace(" ",'_')+'_isNan'
    cd_train[varname]=np.where(pd.isnull(cd_train[col]),1,0)
    cd_train.drop([col],1,inplace=True)
    cd_test[varname]=np.where(pd.isnull(cd_test[col]),1,0)
    cd_test.drop([col],1,inplace=True)
    

In [ ]:
cd_train

In [13]:
for col in cd_train.select_dtypes(['object']).columns:
    print(col,':',cd_train[col].nunique())

Product : 12
Issue : 95
Company : 3276
State : 62
ZIP code : 25962
Submitted via : 6
Company response to consumer : 7
Timely response? : 2
Consumer disputed? : 2


In [14]:
for col in ['ZIP code','Company']:
    cd_train.drop([col],1,inplace=True)
    cd_test.drop([col],1,inplace=True)

In [15]:
cd_train['Consumer disputed?']=np.where(cd_train['Consumer disputed?']=="Yes",1,0)

In [16]:
k=cd_train['Issue'].value_counts()

In [17]:
k

Loan modification,collection,foreclosure    80302
Incorrect information on credit report      58527
Loan servicing, payments, escrow account    51403
Cont'd attempts collect debt not owed       36367
Account opening, closing, or management     23568
Disclosure verification of debt             16235
Communication tactics                       15312
Deposits and withdrawals                    14721
Application, originator, mortgage broker    11201
Billing disputes                             9600
Credit reporting company's investigation     9492
Other                                        9442
Managing the loan or lease                   8905
Problems caused by my funds being low        7758
False statements or representation           7074
Unable to get credit report/credit score     7060
Dealing with my lender or servicer           6460
Improper contact or sharing of info          6182
Problems when you are unable to pay          5921
Settlement process and costs                 5834


In [18]:
for val in k.axes[0][0:10]:
    varname='Issue_'+val.replace(',','_').replace(' ','_')
    cd_train[varname]=np.where(cd_train['Issue']==val,1,0)
    cd_test[varname]=np.where(cd_test['Issue']==val,1,0)
del cd_train['Issue']
del cd_test['Issue']

In [19]:
for col in cd_train.select_dtypes(['object']).columns:
    print(col,':',cd_train[col].nunique())

Product : 12
State : 62
Submitted via : 6
Company response to consumer : 7
Timely response? : 2


In [20]:
k=cd_train['State'].value_counts()

In [21]:
k

CA    70001
FL    46089
TX    35895
NY    32750
GA    21258
NJ    19369
PA    17109
IL    16947
VA    15613
MD    15281
OH    14831
NC    13347
MI    12377
AZ    10691
WA    10024
MA     9584
CO     8231
TN     7383
MO     6221
SC     6000
NV     5838
OR     5698
MN     5587
CT     5582
IN     5303
WI     5259
AL     4950
LA     4776
KY     3419
OK     3172
      ...  
DE     2510
NH     2404
NM     2389
KS     2370
MS     2195
AR     2075
IA     1947
ID     1684
ME     1663
HI     1658
RI     1632
NE     1497
WV     1229
PR     1140
VT      804
MT      738
SD      631
AK      552
WY      459
ND      417
AE      195
AP      132
VI      121
GU       60
FM       30
MH       25
MP       15
AS       14
PW        9
AA        8
Name: State, Length: 62, dtype: int64

In [22]:
for val in k.axes[0][0:15]:
    varname='State_'+val.replace(',','_').replace(' ','_')
    cd_train[varname]=np.where(cd_train['State']==val,1,0)
    cd_test[varname]=np.where(cd_test['State']==val,1,0)
del cd_train['State']
del cd_test['State']

In [23]:
for col in ['Product','Submitted via','Company response to consumer','Timely response?']:
    
    temp=pd.get_dummies(cd_train[col],prefix=col,drop_first=True)
    cd_train=pd.concat([temp,cd_train],1)
    cd_train.drop([col],1,inplace=True)
    
    temp=pd.get_dummies(cd_test[col],prefix=col,drop_first=True)
    cd_test=pd.concat([temp,cd_test],1)
    cd_test.drop([col],1,inplace=True)

In [24]:
x=cd_train.drop(['Consumer disputed?','Complaint ID'],1)
y=cd_train['Consumer disputed?']

In [ ]:
x

In [ ]:
#cd_test.isnull().sum()

#remove data column from train and test dataframe
del cd_train['data']
del cd_test['data']

### use random search model

In [25]:
from sklearn import tree
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [26]:
clf = RandomForestClassifier()

param_dist = {"n_estimators":[100,200,300,500,700,1000],
              "max_features": [5,10,20,25,30,35],
              "bootstrap": [True, False],
              'class_weight':[None,'balanced'], 
                'criterion':['entropy','gini'],
                'max_depth':[None,5,10,15,20,30,50,70],
                'min_samples_leaf':[1,2,5,10,15,20], 
                'min_samples_split':[2,5,10,15,20]
                  }

n_iter_search = 1

In [27]:
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=n_iter_search,scoring='roc_auc',cv=3, verbose=1)

In [28]:
random_search.fit(x, y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 43.7min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=1, n_jobs=1,
          param_distributions={'n_estimators': [100, 200, 300, 500, 700, 1000], 'max_features': [5, 10, 20, 25, 30, 35], 'bootstrap': [True, False], 'class_weight': [None, 'balanced'], 'criterion': ['entropy', 'gini'], 'max_depth': [None, 5, 10, 15, 20, 30, 50, 70], 'min_samples_leaf': [1, 2, 5, 10, 15, 20], 'min_samples_split': [2, 5, 10, 15, 20]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', 

In [29]:
random_search.best_estimator_

RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=15, max_features=35,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=10, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [30]:
##Report function
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [31]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.618 (std: 0.00093)
Parameters: {'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 35, 'max_depth': 15, 'criterion': 'entropy', 'class_weight': None, 'bootstrap': False}



In [34]:
new_rf = RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=15, max_features=35,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=10, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=1, oob_score=False, random_state=None,
            verbose=1, warm_start=False)

In [35]:
new_rf.fit(x,y)

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed: 18.2min finished


RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=15, max_features=35,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=10, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=1, oob_score=False, random_state=None,
            verbose=1, warm_start=False)

In [39]:
x.shape

(478421, 55)

In [41]:
x_test=cd_test.drop(['Complaint ID'],1)

In [42]:
x_test.shape

(119606, 55)

In [44]:
new_rf.predict(x_test)

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   11.2s finished


array([0, 0, 0, ..., 0, 0, 0])

In [45]:
prediction=np.where(new_rf.predict(cd_test.drop(['Complaint ID'],1))==1,"Yes","No")
submission=pd.DataFrame(list(zip(cd_test['Complaint ID'],list(prediction))),
                       columns=['Complaint ID','Consumer disputed?'])

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    9.8s finished


In [46]:
submission.head(4)

,Complaint ID,Consumer disputed?
0,675956,No
1,1858795,No
2,32637,No
3,1731374,No


In [47]:
submission.to_csv('sample_submission.csv',index=False)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
params={'class_weight':['balanced',None],
 'penalty':['l1','l2'],
 'C':np.linspace(0.01,50,10)}

In [ ]:
clf=LogisticRegression()

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_search=GridSearchCV(clf,param_grid=params,cv=5,scoring="roc_auc")

In [ ]:
x.shape

In [ ]:
grid_search.fit(x,y)

In [ ]:
grid_search.best_estimator_

In [ ]:
logr=grid_search.best_estimator_

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


In [ ]:
report(grid_search.cv_results_,5)

In [ ]:
logr.fit(x,y)

In [ ]:
logr

In [ ]:
cd_test.shape

In [ ]:
x.shape

In [ ]:
test_score = logr.predict_proba(cd_test)[:,1]

In [ ]:
clf.fit(x,y)

In [ ]:
prediction=np.where(clf.predict(cd_test.drop(['Complaint ID'],1))==1,"Yes","No")
submission=pd.DataFrame(list(zip(cd_test['Complaint ID'],list(prediction))),
                       columns=['Complaint ID','Consumer disputed?'])

In [ ]:
submission.head(4)

In [ ]:
submission.to_csv('sample_submission.csv',index=False)

This submission will get you auc score of approx 0.50, slightly less than whats required for passing the course. You'll have to make changes 